## Data Preparation

In [2]:
# import necessary modules/libraries

import numpy as  np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import pearsonr

from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=0.7)
sns.set_palette("Greens_r")
#set_palette("Set1", 8, .75) # makes plot lines red

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.preprocessing import Imputer # to impute missing data

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

%matplotlib inline

## Feature Engineering

In [3]:
df = pd.read_csv('house_price_prediction.csv') # read the data

df['date'] = pd.to_datetime(df['date']) # change date col to datetime
df['month'] = df['date'].dt.month # creating month feature
df['year'] = df['date'].dt.year  # all 2014 so can drop
df[['waterfront', 'condition']] = df[['waterfront', 'condition']].astype('category') # change data types to categorical
df['state'] = df['statezip'].apply(lambda x: x.split(' ')[0]) # split statezip into state & zipcode
df['zipcode'] = df['statezip'].apply(lambda x: int(x.split(' ')[1]))
df['total_sqft'] = df.sqft_living + df.sqft_lot # creating total_sqft feature: sqft_above + sqft_lot
df['street_name'] = df['street'].str.strip().str.lstrip('-0123456789').str.strip().str.lower()
df['price_is0'] = df['price'] <= 0 # create column of boolean arrays with 1 for price == $0 & 0 for price != $0
df['renov_date_is0'] = df['yr_renovated'] <= 0 # create column of boolean arrays with 1 for yr_renov == 0 & 0 for others

# replacing 0s with the mean bedroom & bathroom values
df["bedrooms"].replace({0: round(df["bedrooms"].mean(), 0)}, inplace=True)
df["bathrooms"].replace({0: round(df["bathrooms"].mean(), 0)}, inplace=True)

In [4]:
for c in ['statezip', 'country', 'state', 'street']:
    try:
        del df[c]
    except KeyError:
        pass

In [5]:
#df['month'] = df['month'].astype(int)
# remove `month` from here and do get_dummies on it after plotting ecdf
df = pd.get_dummies(df, columns=['month', 'zipcode', 'street_name', 'city'])

In [6]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,city_SeaTac,city_Seattle,city_Shoreline,city_Skykomish,city_Snoqualmie,city_Snoqualmie Pass,city_Tukwila,city_Vashon,city_Woodinville,city_Yarrow Point
0,2014-05-02,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,...,0,0,1,0,0,0,0,0,0,0
1,2014-05-02,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,...,0,1,0,0,0,0,0,0,0,0
2,2014-05-02,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,2014-05-02,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
4,2014-05-02,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


**Further feature engineering**

In [7]:
df_no_outs = df[df.price < 10000000].copy() # df with price outliers removed
df_no_zeros = df_no_outs[df_no_outs["price"] != 0].copy() # df with $0 prices removed

df2 = df.copy()
df2['yr_built_is0'] = df2['yr_built'] <= 0
df2['renovation_age'] = df2['year'] - df2['yr_renovated']
df2['house_age'] = df2['year'] - df2['yr_built']
df2['sqft_lot'] = df2[df2['sqft_lot'] < 600000]
df2_no_outs = df[df.price < 10000000].copy() # df2 with price outliers removed
df2_no_zeros = df_no_outs[df_no_outs["price"] != 0].copy() # df2 with $0 prices removed

## Statistical Data Analysis

In [8]:
'''
meanprice_bedrooms = df_no_zeros.groupby('bedrooms')['price'].mean()
medianprice_bedrooms = df_no_zeros.groupby('bedrooms')['price'].median()

meansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].mean()
mediansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].median()

may_price = df_no_zeros[df_no_zeros.month==5]['price']
june_price = df_no_zeros[df_no_zeros.month==6]['price']
july_price = df_no_zeros[df_no_zeros.month==7]['price']

def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    n = len(data)

    x = np.sort(data)

    y = np.arange(1, n+1) / n

    return x, y

# Compute ECDFs
x_5, y_5 = ecdf(may_price)
x_6, y_6 = ecdf(june_price)
x_7, y_7 = ecdf(july_price)
'''

'\nmeanprice_bedrooms = df_no_zeros.groupby(\'bedrooms\')[\'price\'].mean()\nmedianprice_bedrooms = df_no_zeros.groupby(\'bedrooms\')[\'price\'].median()\n\nmeansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].mean()\nmediansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].median()\n\nmay_price = df_no_zeros[df_no_zeros.month==5][\'price\']\njune_price = df_no_zeros[df_no_zeros.month==6][\'price\']\njuly_price = df_no_zeros[df_no_zeros.month==7][\'price\']\n\ndef ecdf(data):\n    """Compute ECDF for a one-dimensional array of measurements."""\n    n = len(data)\n\n    x = np.sort(data)\n\n    y = np.arange(1, n+1) / n\n\n    return x, y\n\n# Compute ECDFs\nx_5, y_5 = ecdf(may_price)\nx_6, y_6 = ecdf(june_price)\nx_7, y_7 = ecdf(july_price)\n'

### Feature Importance

In [9]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

def feature_importance(X, y, model='reg'):
    score_func = {'reg': f_regression}

    # Score each of the features
    bestfeatures = SelectKBest(score_func=score_func[model], k='all')
    fit = bestfeatures.fit(X, y)

    # Organize and return the scores
    featureScores = pd.DataFrame([X.columns, fit.scores_]).T
    featureScores.columns = ['Feature', 'Score']
    return featureScores.sort_values('Score', ascending=False).set_index('Feature') 

### Hypothesis Test I

**Ho**: Prices of houses with many bedrooms and a few bedrooms are equal.  
**Ha**: Prices of houses with many bedrooms and a few bedrooms are different.

t-test:
I'll compare 2 groups: one with `bedrooms` greater than or equal to 4, and another with less than 4.

In [10]:
many_bedrooms = df_no_outs[df_no_outs['bedrooms'] >= 4]['price']
few_bedrooms = df_no_outs[df_no_outs['bedrooms'] < 4]['price']
stats.ttest_ind(many_bedrooms, few_bedrooms)

Ttest_indResult(statistic=20.612406446603853, pvalue=2.3169170411918228e-90)

#### Result
**t-stat**: 20.6  
**p-val**: 2.3e-90 < 0.05  
We reject the null hypothesis that houses with many bedrooms are priced around the same as houses with fewer bedrooms.

### Hypothesis Test II

**Ho**: Houses with waterfronts are priced the same as houses without waterfronts.  
**Ha**: Houses with waterfronts are priced differently than houses without waterfronts.

In [11]:
waterfront = df_no_outs[df_no_outs['waterfront'] == 1]['price']
no_waterfront = df_no_outs[df_no_outs['waterfront'] == 0]['price']
stats.ttest_ind(waterfront, no_waterfront)

Ttest_indResult(statistic=14.441521571804724, pvalue=2.8817562762538972e-46)

#### Result
**t-stat**: 14.4  
**p-val**: 2.9e-46 < 0.05  
We reject the null hypothesis that houses with waterfronts are priced the same as houses without waterfronts.

# Data Modeling

## Data Preprocessing

In [12]:
# dataframe with both zero prices and outliers.
X = df.drop(['price', 'date', 'year'], axis='columns')
y = df["price"]
# splitting data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# dataframe without price outliers
X_no_outs = df_no_outs.drop(['price', 'date', 'year'], axis='columns')
y_no_outs = df_no_outs["price"]
X_train_no_outs, X_test_no_outs, y_train_no_outs, y_test_no_outs = train_test_split(X_no_outs, y_no_outs, test_size = 0.3, random_state=42)

# dataframe without $0 price values
X_no_zeros = df_no_zeros.drop(['price', 'date', 'year'], axis='columns')
y_no_zeros = df_no_zeros["price"]
X_train_no_zeros, X_test_no_zeros, y_train_no_zeros, y_test_no_zeros = train_test_split(X_no_zeros, y_no_zeros, test_size = 0.3, random_state=42)

# dataframe with a parabolic transformation on the `bedrooms` feature
X_transformed = df_no_zeros.drop(['price', 'date', 'year'], axis='columns')
X_transformed['bedrooms_squared'] = X['bedrooms']**2  # parabolic transformation of bedrooms
y_transformed = df_no_zeros['price']
X_train_transformed, X_test_transformed, y_train_transformed, y_test_transformed = train_test_split(X_transformed, y_transformed, test_size = 0.3, random_state=42)

# dataframe with both zero prices and outliers.
X2 = df2.drop(['price', 'date', 'year'], axis='columns')
y2 = df2["price"]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.3, random_state=42)

# dataframe without price outliers
X2_no_outs = df2_no_outs.drop(['price', 'date', 'year'], axis='columns')
y2_no_outs = df2_no_outs["price"]
X2_train_no_outs, X2_test_no_outs, y2_train_no_outs, y2_test_no_outs = train_test_split(X2_no_outs, y2_no_outs, test_size = 0.3, random_state=42)

# dataframe without $0 price values
X2_no_zeros = df2_no_zeros.drop(['price', 'date', 'year'], axis='columns')
y2_no_zeros = df2_no_zeros["price"]
X2_train_no_zeros, X2_test_no_zeros, y2_train_no_zeros, y2_test_no_zeros = train_test_split(X2_no_zeros, y2_no_zeros, test_size = 0.3, random_state=42)

# dataframe with a parabolic transformation on the `bedrooms` feature
X2_transformed = df2_no_zeros.drop(['price', 'date', 'year'], axis='columns')
X2_transformed['bedrooms_squared'] = X2_transformed['bedrooms']**2  # parabolic transformation of bedrooms
y2_transformed = df2_no_zeros['price']
X2_train_transformed, X2_test_transformed, y2_train_transformed, y2_test_transformed = train_test_split(X2_transformed, y2_transformed, test_size = 0.3, random_state=42)


In [13]:
# 4 training sets and test sets
# 3 algorithms

train_test_sets = [
    [X_train, X_test, y_train, y_test],
    [X_train_no_outs, X_test_no_outs, y_train_no_outs, y_test_no_outs],
    [X_train_no_zeros, X_test_no_zeros, y_train_no_zeros, y_test_no_zeros],
    [X_train_transformed, X_test_transformed, y_train_transformed, y_test_transformed],
    [X2_train, X2_test, y2_train, y2_test],
    [X2_train_no_outs, X2_test_no_outs, y2_train_no_outs, y2_test_no_outs],
    [X2_train_no_zeros, X2_test_no_zeros, y2_train_no_zeros, y2_test_no_zeros],
    [X2_train_transformed, X2_test_transformed, y2_train_transformed, y2_test_transformed],
]

train_test_dicts = []
for i, sets in enumerate(train_test_sets):
    d = dict(zip('X_train X_test y_train_true y_test_true'.split(), sets))
    train_test_dicts.append(d)

algorithms = [LinearRegression, Ridge, RandomForestRegressor]
names = ['linear regression', 'ridge regression ', 'random forest    ']

In [14]:
'''
def calc_train_error(X_train, y_train, model):
    #returns in-sample error for already fit model.
    y_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    rmse = np.sqrt(mse)
    return rmse
    
def calc_test_error(X_test, y_test, model):
    #returns out-of-sample error for already fit model.
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse
    '''

def calc_rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean(y_true - y_pred)**2)
    return rmse
    
def fit_and_score(X_train, y_train_true, X_test, y_test_true, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error, and their accuracy score'''
    
    model.fit(X_train, y_train_true)
    y_train_pred = model.predict(X_train)
    train_error = calc_rmse(y_true=y_train_true, y_pred=y_train_pred)
    y_test_pred = model.predict(X_test)
    test_error = calc_rmse(y_true=y_test_true, y_pred=y_test_pred)
    #print(y_train_true.shape, y_train_pred.shape)
    
    train_score = model.score(X_train, y_train_true)
    test_score = model.score(X_test, y_test_true)
    
    return train_error, test_error, train_score, test_score, y_train_pred, y_test_pred

### Choosing alpha for Ridge Regression

In [ ]:
alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1]

for sets in train_test_dicts:
    print('')
    for alpha in alphas:
        model = Ridge(alpha = alpha)
        train_error, test_error, train_score, test_score, y_train_pred, y_test_pred = fit_and_score(model=model, **sets)
        #train_error, test_error, train_score, test_score, y_pred = fit_and_score(sets[0], sets[2],  sets[1], sets[3], model)
        print('alpha: {} | test_score: {} | test_error: {} | test/train error: {}'.format(alpha, test_score, test_error, round(test_error/train_error, 1)))


For the 1st & 2nd datasets (df3 & df_no_outs), the best alpha seems to be alpha=1.  
For the 3rd and 4th datasets (df_no_zeros & df_transformed), alpha=0.0001 seems best.

### Choosing parameters for Random Forest

In [ ]:
max_depths = [None, 10, 100]
n_estimators = [1, 10, 100, 200]

for sets in train_test_dicts:
    print('')
    for max_depth in max_depths:
        for n in n_estimators:
            model = RandomForestRegressor(max_depth=max_depth , n_estimators=n)
            train_error, test_error, train_score, test_score, y_train_pred, y_test_pred = fit_and_score(model=model, **sets)
            train_error, test_error, train_score, test_score = round(train_error,3), round(test_error,3), round(train_score,3), round(test_score,3)
            print('max_depth: {} | n_estimators: {} | test_score: {} | test_error: {} | test/train error: {}'.format(max_depth, n, test_score, test_error, round(test_error/train_error, 1)))


Looks like the best values for parameters `max_depth` & `n_estimators` are None & 200 respectively.

In [17]:
'''lr_train_score = []
lr_test_score = []
lr_train_error = []
lr_test_error = []

ridge_train_score = []
ridge_test_score = []
ridge_train_error = []
ridge_test_error = []

rf_train_score = []
rf_test_score = []
rf_train_error = []
rf_test_error = []'''

train_scores = []
test_scores = []
train_errors = []
test_errors = []
y_train_preds = []
y_test_preds = []

predictions_dict = dict()

for sets in train_test_dicts:
    for model_class, hyperparams in zip(
            [LinearRegression, Ridge, RandomForestRegressor], 
            [{}, dict(alpha=0.0001), dict(max_depth=None, n_estimators=200)]
            ):
        model = model_class(**hyperparams)
        
        train_error, test_error, train_score, test_score, y_train_pred, y_test_pred = fit_and_score(model=model, **sets)
        #train_error, test_error, train_score, test_score, y_train_pred, y_test_pred = round(train_error,1), round(test_error,1), round(train_score,0), round(test_score,0), round(y_train_pred,0), round(y_test_pred,0)

        train_errors.append(train_error)
        test_errors.append(test_error)
        train_scores.append(train_score)
        test_scores.append(test_score)
        y_train_preds.append(y_train_pred)
        y_test_preds.append(y_test_pred)

        '''for i in range(24):
            if i < 9:
                predictions_dict['lr_pred{}'.format(i)] = y_test_pred
            elif i>=9 and i < 16:
                predictions_dict['ridge_pred{}'.format(i)] = y_test_pred
            else:
                predictions_dict['rf_pred{}'.format(i)] = y_test_pred'''

        

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [18]:
dataTypeDict = dict(df.dtypes)
print('Data type of each column :')
print(dataTypeDict)

Data type of each column :
{'date': dtype('<M8[ns]'), 'price': dtype('float64'), 'bedrooms': dtype('float64'), 'bathrooms': dtype('float64'), 'sqft_living': dtype('int64'), 'sqft_lot': dtype('int64'), 'floors': dtype('float64'), 'waterfront': CategoricalDtype(categories=[0, 1], ordered=False), 'view': dtype('int64'), 'condition': CategoricalDtype(categories=[1, 2, 3, 4, 5], ordered=False), 'sqft_above': dtype('int64'), 'sqft_basement': dtype('int64'), 'yr_built': dtype('int64'), 'yr_renovated': dtype('int64'), 'year': dtype('int64'), 'total_sqft': dtype('int64'), 'price_is0': dtype('bool'), 'renov_date_is0': dtype('bool'), 'month_5': dtype('uint8'), 'month_6': dtype('uint8'), 'month_7': dtype('uint8'), 'zipcode_98001': dtype('uint8'), 'zipcode_98002': dtype('uint8'), 'zipcode_98003': dtype('uint8'), 'zipcode_98004': dtype('uint8'), 'zipcode_98005': dtype('uint8'), 'zipcode_98006': dtype('uint8'), 'zipcode_98007': dtype('uint8'), 'zipcode_98008': dtype('uint8'), 'zipcode_98010': dtype('

In [ ]:
'''
# constructing table contents 

features = []
for i in range(24):
    features.append('all except date, price, street, city, state')

features_dict = {key: value for key, value in enumerate(features) }


lr = 'linear regression'
ridge = 'ridge regression'
rf = 'random forest'

models = [(lr,)*8, (ridge,)*8, (rf,)*8]


test_error_list = []
for i in range(8):
    test_error_list.append(lr_test_error[i])
for i in range(8):
    test_error_list.append(ridge_test_error[i])
for i in range(8):
    test_error_list.append(rf_test_error[i])
'''

In [ ]:
'''
features1 = 'all except date, price, street, city, state'
features2 = 'all except date, price, street, city, state'
features3 = 'all except date, price, street, city, state'
features4 = 'all except date, price, street, city, state'
features5 = 'all except date, price, street, city, state'
features6 = 'all except date, price, street, city, state'
features7 = 'all except date, price, street, city, state'
features8 = 'all except date, price, street, city, state'
features9 = 'all except date, price, street, city, state'
features10 = 'all except date, price, street, city, state'
features11 = 'all except date, price, street, city, state'
features12 = 'all except date, price, street, city, state'
'''


content = {''''features': [features1, features2, features3, features4, features5, features6, features7, features8, 
                        features9, features10, features11, features12],''' 
           'features': [features_dict.values()],
           'model': ['linear regression', 'linear regression', 'linear regression', 'linear regression',
                     'ridge regression', 'ridge regression', 'ridge regression', 'ridge regression', 
                     'random forest', 'random forest', 'random forest', 'random forest'], 
           'test_error': [lr_test_error[0], lr_test_error[1], lr_test_error[2], lr_test_error[3], 
                    ridge_test_error[0], ridge_test_error[1], ridge_test_error[2], ridge_test_error[3], 
                    rf_test_error[0], rf_test_error[1], rf_test_error[2], rf_test_error[3]],
           'train_error': [lr_train_error[0], lr_train_error[1], lr_train_error[2], lr_train_error[3], 
                    ridge_train_error[0], ridge_train_error[1], ridge_train_error[2], ridge_train_error[3], 
                    rf_train_error[0], rf_train_error[1], rf_train_error[2], rf_train_error[3]],
           'test_score': [lr_test_score[0], lr_test_score[1], lr_test_score[2], lr_test_score[3], 
                     ridge_test_score[0], ridge_test_score[1], ridge_test_score[2], ridge_test_score[3],  
                     rf_test_score[0], rf_test_score[1], rf_test_score[2], rf_test_score[3]],
           'train_score': [lr_train_score[0], lr_train_score[1], lr_train_score[2], lr_train_score[3],
                             ridge_train_score[0], ridge_train_score[1], ridge_train_score[2], ridge_train_score[3],
                             rf_train_score[0], rf_train_score[1], rf_train_score[2], rf_train_score[3]],
           'description': ['no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation', 
                           'no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation', 
                           'no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation', 
                            ],
           'price outs removed': ['no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes']}

table = pd.DataFrame(content, columns=['model', 'test_score', 'train_score', 'test_error', 'train_error', 'price outs removed', 'description', 'features'], index=[1,2,3,4,5,6,7,8,9,10,11,12])
table.sort_values('test_score', ascending=False)

Model description:
* Models described as having no transformations are those which did not have their `bedrooms` feature undergo parabolic transformation.
* Models with zero prices removed are those with zero values in the `price` feature and price outliers removed.
* Models with parabolic transformation are based on data with price outliers and zero prices removed, and have their `bedrooms` feature transformed.

From the table above we can see that the best model is the Random Forest with 66% accuracy. That model only has the price outliers removed but not the zero prices. There were also no transformations on the features.

**Analysing the coefficients**

Let's take a look at the coefficients of the best performing ridge regression model that has its `bedrooms` feature transformed.

In [ ]:
coefs = pd.DataFrame(ridge.coef_, index=list(X_transformed), columns=['Coef'])
coefs[:18]

The coefficient of a term represents the change in the mean response for one unit of change in that term. If the coefficient is positive, as the term increases, the mean value of the response increases. If the coefficient is negative, as the term increases, the mean value of the response decreases. So for example we see than the coefficient of `sqft_living` is 180. This means that for every 1 sqft increase on a house, the price would increase by \\$180.

I was surprised to see `bedrooms` have a negative coeficient, since my assumption was that when the number of bedrooms increase, so does the price. However, we saw previously with a `bedrooms` and `price` scatterplot that price increases with number of bedrooms only for homes with up to 5 bedrooms. The price then starts to decrease for homes with 6 bedrooms or more.  
Notice also `bedrooms_squared` has a positive coeficient. This feature is `bedrooms` with a parabolic transformation. Since this feature has adjusted for `bedrooms` shape, it makes sense that it is positive.

Not only is the coefficient of `sqft_lot` negative, but it's also a small number, indicating that its affect on  price is miniscule.

In [ ]:
# coefs of fitted model & predicted model
#coefs = pd.DataFrame({'coef_actual': ridge.coef_, 'coef_pred': predictions_dict['ridge_pred3']}, index=list(X_transformed))

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(y_test_transformed, 'bo', alpha=0.3, label='actual prices')
ax.plot(predictions_dict['lr_pred3'], 'go', alpha=0.2, label='predicted prices')
#ax.set_xlabel('')
ax.set_ylabel('price')
ax.legend()
plt.show()

In [ ]:
plt.scatter(y_test_transformed, predictions_dict['lr_pred3'], s=20)
plt.title('Scatter plot of actual and predicted prices')
plt.xlabel('actual prices')
plt.ylabel('predicted prices')

We see in the above plot that the predicted prices are somewhat close to the actual prices, but not very close. The best case scenario would be for the scatter plots to follow a perfectly straight diagonal line, indicating that our model perfectly predicted the house prices.

Histogram of the residuals of the best performing linear regression model.

In [ ]:
plt.subplots(figsize=(8,4))
plt.hist(y_test_transformed - predictions_dict['lr_pred3'], bins=20)
plt.title('Histogram of residuals')
plt.show()

Ideally a residual histogram would follow a normal distribution, and that would tell us that we've chosen an appropriate model type. However, the above plot does not closely follow a normal distribution, which is another indication that a linear regression model is not best for my dataset.

### CI around my prediction accuracy

calculate standard error RMSE
..

In [ ]:
# RMSE == standard deviation of errors
rf_test_error

In [ ]:
one_sd = min(rf_test_error)
two_sd = 2*one_sd
three_sd = 3*one_sd

#### 68% Confidence

In [ ]:
min_error = 0 - one_sd
max_error = 0 + one_sd

print('I can be 68% confident that the error in the home price predictions will be between {} and {}.'.format(min_error, max_error))

#### 95% Confidence

In [ ]:
min_error = 0 - two_sd
max_error = 0 + two_sd

print('I can be 95% confident that the error in the home price predictions will be between {} and {}.'.format(min_error, max_error))

### Result Analysis

The following analysis is based on my 95% confidence that the error in the home price predictions is between -429250 and -429250:

If I've predicted a house to be $\$$500,000 and someone were to buy the house at that price, the worst loss would be \\$429250 and the highest gain would be \\$429250. That is, the least the house would be worth is \\$70,750 (500,000-429250), and the most the house would be worth is \\$929,250 (500,000+429250).

Let's say my boss wants to buy a house at below my estimate in the hopes of flipping it, i.e., selling the house at around double the price at which he acquired it. With a house that I have estimated to be \\$500,000, if he purchases it at \\$300,000 for example, 3 things can happen:

1. My estimation is correct and he would make a profit of \\$200,000 by selling it for \\$500,000.  
2. The house is actually worth \\$70,750 and he ends up buying at a higher price than what the house is worth, losing \\$229,250 on the purchase, and resulting in a 24\% loss.
3. The house is actually worth \\$929,250 and he ends up buying \\$629,250 below the house value, resulting in a 309\%  profit if he were to sell it at \\$929,250.

**What happens if my boss purchases 100 homes?**

In [ ]:
from math import sqrt

np.random.seed(47)
n = 100 # length of sample

# function to sample the actual and predicted prices of 100 homes at a time.
def draw_bs_reps(y_test, y_pred):
    bs_error = np.random.choice(y_test - y_pred, n) # price sold - price bought == profit (gain or loss)
    
    error_mean = np.sum(bs_error) / len(bs_error) # avg gain or loss on n transactions
    return error_mean

In [ ]:
# conducting 1000 experiments of purchasing 100 homes
error_means = np.array([draw_bs_reps(y_test_transformed, predictions_dict['rf_pred3']) for i in range(1000)]) # Random Forest model
error_means = np.round(error_means, 2)
error_means_mean = np.mean(error_means) # mean of means
print('mean of means: {}'.format(int(error_means_mean)))

error_means_std = sqrt(np.sum((error_means-error_means_mean)**2) / (1000-1)) # standard deviation of means
print('std of means: {}'.format(int(error_means_std)))

# confidence interval
min_error_100 = int(error_means_mean - 2*error_means_std)
max_error_100 = int(error_means_mean + 2*error_means_std)

print('')
print('The above result means that if I calculate the error between the true/historical price and my predicted price ' 
      'of 100 homes, I can be 95% confident that the error of the mean of average error in the home price ' 
      'predictions will be between {:,} and {:,}.'.format(min_error*100, max_error*100))
print('')
print('In other words, if my boss was to buy 100 homes at my predicted price and sell them at their true price, ' 
      'I can be 95% confident that the most she would lose is ${:,} ' 
      'and the most she would gain is ${:,}.'.format(-1*min_error_100*100, max_error_100*100))

Below is a histogram of the distribution of purchasing 100 homes at my predicted prices.

In [ ]:
plt.hist(error_means, bins=20)
plt.plot()
print(min_error_100, max_error_100)

It shows that the mode is shifted to the positive, which is good because it means we're profiting. I conducted this experiment a few more times to make sure the shape is real.

What happens if my boss purchases homes $1000 below my predicted prices?

In [ ]:
# running the same experiment as before, with $1000 deducted from my predicted prices

np.random.seed(47)
n = 100 # length of sample
N_rep = 1000 # number of times to bootstrap/resample

def draw_bs_reps(y_test, y_pred):
    bs_error = np.random.choice(y_test - (y_pred-1000), n) # deduct 1000 from each predicted price
    
    error_mean = np.sum(bs_error) / len(bs_error) # avg gain or loss on n transactions
    return error_mean

In [ ]:
error_means = np.array([draw_bs_reps(y_test_transformed, predictions_dict['rf_pred3']) for i in range(1000)]) # Random Forest model
error_means = np.round(error_means, 2)
error_means_mean = np.mean(error_means) # mean of means
print('mean of means: {}'.format(int(error_means_mean)))

error_means_std = sqrt(np.sum((error_means-error_means_mean)**2) / (1000-1)) # standard deviation of means
print('std of means: {}'.format(int(error_means_std)))

# confidence interval
min_error_1000 = int(error_means_mean - 2*error_means_std)
max_error_1000 = int(error_means_mean + 2*error_means_std)

print('')
print('If my boss was to buy 100 homes at $1000 less than my predicted price and sell them at their true price, ' 
      'I can be 95% confident that the most she would lose is ${:,} ' 
      'and the most she would gain is ${:,}.'.format(-1*min_error_1000*100, max_error_1000*100))

When purchasing 100 homes at \\$1000 below my predicted price, the most my boss would lose decreases by \\$ and the most she would gain increases by \\$.  
This is clearly a better option, and since finding homes that are selling \\$1000 below their true value is practical, I would recommend this plan to my boss.

### Result

Other experiments:
* experiments with other models
* experiemnts with predicted prices reduced by \\$5,000, \\$10,000, \\$50,000

When running experiments using my other models, the Random Forest model resulted in the most profit, which was expected since the model had the least error in its prediction.

I ran experiments with \\$5,000, \\$10,000, \\$50,000 deducted from the predicted prices. 
This means that my boss would purchase 100 homes with \\$5,000, \\$10,000, and \\$50,000 below my predicted prices. 
The result was that these 3 experiments produced better profits and less losses than the scenario where my boss would purchase homes at \\$1000 less than my predicted prices. However, finding 100 houses at those reduced prices would be difficult. Therefore, I would reccomend focusing on finding homes with the \\$1000 reductions.